# 变分影子量子学习

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概览

本教程我们将讨论变分影子量子学习（variational shadow quantum learning, VSQL）[1] 的原理，以及如何使用 VSQL 来完成一个**二分类**任务。VSQL 是一个在监督学习框架下的量子–经典混合算法。它使用了参数化量子电路（parameterized quantum circuit, PQC）和经典影子（classical shadow），和通常使用的变分量子算法（variational quantum alogorithm, VQA）不同的是，VSQL 只从子空间获取局部特征，而不是从整个希尔伯特空间获取特征。

### 背景

我们考虑一个 $k$ 分类问题。输入是一个由 $N$ 个带标签的数据点组成的集合 $D=\left\{(\mathbf{x}^i, \mathbf{y}^i)\right\}_{i=1}^{N}$，这里的 $\mathbf{x}^i\in\mathbb{R}^{m}$ 表示数据点，$\mathbf{y}^i$ 是这个数据点的标签。**学习过程的目的在于训练一个能尽可能准确预测所有数据点标签的模型 $\mathcal{F}$。** 我们需要注意，$\mathbf{y}^i$ 是一个长度为 $k$ 的独热向量（one-hot vector），将标签值表示为独热向量是机器学习领域中的常规做法。我们举一个例子，当 $k=3$ 时，$\mathbf{y}^{a}=[1, 0, 0]^\text{T}$、$\mathbf{y}^{b}=[0, 1, 0]^\text{T}$ 和 $\mathbf{y}^{c}=[0, 0, 1]^\text{T}$ 分别表示第 $a$ 个、第 $b$ 个和第 $c$ 个数据点属于第一类、第二类和第三类。  
在 VSQL 中，$\mathcal{F}$ 由两部分组成：参数化局部量子电路和经典全连接神经网络（fully-connected neural network, FCNN）。首先，我们需要预处理经典信息，将经典信息编码到量子态。然后我们以卷积的方式将 $U(\mathbf{\theta})$ 连续作用在局部量子比特上，这里的 $\mathbf{\theta}$ 是参数化局部电路的参数。在这些局部量子比特上进行测量就能得到我们想要的期望值。除此之外，我们还需要一个经典 FCNN 来做数据的后处理。  
我们可以把 VSQL 得到的 $\mathcal{F}$ 的输出写为 $\tilde{\mathbf{y}}^i = \mathcal{F}(\mathbf{x}^i)$。这里的 $\tilde{\mathbf{y}}^i$ 是一个概率分布，$\tilde{y}^i_j$ 表示第 $i$ 个数据点属于第 $j$ 类的概率。为了尽可能地预测到准确的标签，我们将预测标签 $\tilde{\mathbf{y}}^i$ 和实际标签 $\mathbf{y}^i$ 之间的累计差距作为损失函数 $\mathcal{L}$ 进行优化：  

$$
\mathcal{L}(\mathbf{\theta}, \mathbf{W}, \mathbf{b}) = -\frac{1}{N}\sum\limits_{i=1}^{N}\sum\limits_{j=1}^{k}y^i_j\log{\tilde{y}^i_j}, \tag{1}
$$

其中 $\mathbf{W}$ 和 $\mathbf{b}$ 是经典 FCNN 的权重（weight）和偏置（bias）。注意这个损失函数是由交叉熵（cross-entropy）[2] 推导得到的。

### 方案流程  

![pipeline](./figures/vsql-fig-pipeline-cn.png "图1：VSQL 的流程图")
<div style="text-align:center">图1：VSQL 的流程图 </div>

这里我们给出实现 VSQL 的流程。

1. 将经典数据 $\mathbf{x}^i$ 编码到量子态 $\left|\mathbf{x}^i\right>$。
2. 准备一个参数化局部量子电路 $U(\mathbf{\theta})$ 并且初始化它的参数 $\mathbf{\theta}$。
3. 在前几个量子比特上作用 $U(\mathbf{\theta})$，然后通过测量局部可观测量（比如说泡利 $X\otimes X\cdots \otimes X$ 算符）来获取一个局部影子特征。
4. 每次将 $U(\mathbf{\theta})$ 向下移动一个量子比特，重复步骤3直到 $U(\mathbf{\theta})$ 作用到最后一个量子比特上。
5. 将步骤3–4中得到的所有局部影子特征传入经典 FCNN 并通过激活函数得到预测的标签 $\tilde{\mathbf{y}}^i$。对于多分类问题来说，我们使用归一化指数函数 (softmax) 作为激活函数。
6. 重复步骤3–5直到数据集内所有的数据点都经过了处理。然后计算损失函数 $\mathcal{L}(\mathbf{\theta}, \mathbf{W}, \mathbf{b})$。
7. 通过梯度下降等优化方法调整参数 $\mathbf{\theta}$、$\mathbf{W}$ 和 $\mathbf{b}$ 的值，从而最小化损失函数。这样我们就得到了优化后的模型 $\mathcal{F}$。

由于 VSQL 只获取局部影子特征，所以它可以比较容易地在有拓扑连接限制的量子设备上实现。除此之外，因为我们用同一个 $U(\mathbf{\theta})$ 来获取整个电路上的局部影子特征，所以需要训练的参数数量相对于通常使用的变分量子分类器来说大大减少。

## Paddle Quantum 实现

接下来，我们将用 VSQL 来完成手写数字图像二分类的问题。我们使用的数据来源于常用于基准测试的公开数据集 MNIST [3]，其中包含了标签为从 '0' 到 '9' 的十个类别的数据。为了便于展示，我们这里只考虑二分类问题，使用标签为 '0' 和 '1' 的数据。  

首先，导入所需要的语言包：

In [2]:
import time
import numpy as np
import paddle
import paddle.nn.functional as F
from paddle.vision.datasets import MNIST
import paddle_quantum
from paddle_quantum.ansatz import Circuit

### 数据的预处理

每个手写数字图像都是由 $28\times 28$ 个取值在 $[0, 255]$ 之间的灰度像素点组成。我们需要先把这个 $28\times 28$ 的二维矩阵转化为一个长度为784的一维向量 $\mathbf{x}^i$，然后再使用振幅编码将每个 $\mathbf{x}^i$ 编码到有10个量子比特的量子态 $\left|\mathbf{x}^i\right>$。为了进行振幅编码，我们需要归一化每个向量，再在它的尾部补零使每个向量的长度与有10个量子比特的量子态一致。

In [3]:
# 归一化处理向量并在尾部补零
def norm_img(images):
    new_images = [np.pad(np.array(i).flatten(), (0, 240), constant_values=(0, 0)) for i in images]
    new_images = [paddle.to_tensor(i / np.linalg.norm(i), dtype='complex64') for i in new_images]

    return new_images

In [4]:
def data_loading(n_train=1000, n_test=100):
    # 我们使用 PaddlePaddle 提供的 MNIST
    train_dataset = MNIST(mode='train')
    test_dataset = MNIST(mode='test')
    # 选出标签为0和1的数据点
    train_dataset = np.array([i for i in train_dataset if i[1][0] == 0 or i[1][0] == 1], dtype=object)
    test_dataset = np.array([i for i in test_dataset if i[1][0] == 0 or i[1][0] == 1], dtype=object)
    np.random.shuffle(train_dataset)
    np.random.shuffle(test_dataset)
    # 将手写数字图像和标签分开
    train_images = train_dataset[:, 0][:n_train]
    train_labels = train_dataset[:, 1][:n_train].astype('int64')
    test_images = test_dataset[:, 0][:n_test]
    test_labels = test_dataset[:, 1][:n_test].astype('int64')
    # 归一化向量并补零
    x_train = norm_img(train_images)
    x_test = norm_img(test_images)
    # 将标签处理为独热向量
    train_targets = np.array(train_labels).reshape(-1)
    y_train = paddle.to_tensor(np.eye(2)[train_targets])
    test_targets = np.array(test_labels).reshape(-1)
    y_test = paddle.to_tensor(np.eye(2)[test_targets])

    return x_train, y_train, x_test, y_test

### 搭建局部影子电路

接下来我们要搭建电路。在讲电路的细节之前，我们需要说明几个参数：
- $n$：编码后量子态的量子比特数目。
- $n_{qsc}$：量子影子电路的宽度。我们每次只在连续 $n_{qsc}$ 个量子比特上作用 $U(\mathbf{\theta})$。
- $D$：电路的深度，表示 $U(\mathbf{\theta})$ 门中某一层电路重复的次数。

这里我们给出 $n=4$、$n_{qsc}=2$ 时的一个例子：

我们首先在前两个量子比特上作用 $U(\mathbf{\theta})$，并且获取第一个影子特征 $O_1$。

![qubit0](./figures/vsql-fig-qubit0-cn.png "图2：获取第一个影子特征")
<div style="text-align:center">图2： 获取第一个影子特征 </div>

然后我们准备一样的输入态 $\left|\mathbf{x}^i\right>$，在中间两个量子比特上作用 $U(\mathbf{\theta})$，得到第二个影子特征 $O_2$。

![qubit1](./figures/vsql-fig-qubit1-cn.png "图3：获取第二个影子特征")
<div style="text-align:center">图3： 获取第二个影子特征 </div>

最后，我们再准备一个一样的输入态，在最后两个量子比特上作用 $U(\mathbf{\theta})$，得到影子特征 $O_3$。这样我们就处理完了这个数据点！

![qubit2](./figures/vsql-fig-qubit2-cn.png "图4：获取第三个影子特征")
<div style="text-align:center">图4： 获取第三个影子特征 </div>

通常来说，处理一个数据点需要重复以上步骤 $n - n_{qsc} + 1$ 次。有一点需要指出的是，在上面这个例子中我们只使用了一个影子电路，在获取这三个影子特征时我们使用同样的参数 $\mathbf{\theta}$。你可以选择增加影子电路的数量来解决更复杂的问题，这里需要注意的是不同影子电路中的参数 $\mathbf{\theta}$ 不同。  
 
在后面的 MNIST 二分类任务中，我们将使用2–局部影子电路，即 $n_{qsc}=2$。图5展示了这个影子电路的结构。

![2-local](./figures/vsql-fig-2-local.png "图5：$n_{qsc}=2$ 时 $U(\mathbf{\theta})$ 的结构")
<div style="text-align:center">图5：$n_{qsc}=2$ 时 $U(\mathbf{\theta})$ 的结构</div>

为了增强量子电路的表达能力，我们将重复 $D$ 次虚线框中的结构。$U(\mathbf{\theta})$ 的设计并不是唯一的，这里展示的仅仅是一个例子，读者不妨尝试设计自己的电路结构。

In [5]:
# 搭建 U(theta)
def U_theta(n, n_qsc=2, depth=1):
    # 初始化电路
    cir = Circuit(n)
    # 先搭建广义的旋转层
    for i in range(n_qsc):
        cir.rx(qubits_idx=i)
        cir.ry(qubits_idx=i)
        cir.rx(qubits_idx=i)
    # 搭建纠缠层和 Ry 旋转层，重复 D 次
    for repeat in range(1, depth + 1):
        for i in range(n_qsc - 1):
            cir.cnot([i, i + 1])
        cir.cnot([n_qsc - 1, 0])
        for i in range(n_qsc):
            cir.ry(qubits_idx=i)

    return cir

# 滑动电路
def slide_circuit(cir, distance):
    for sublayer in cir.sublayers():
        qubits_idx = np.array(sublayer.qubits_idx)
        qubits_idx = qubits_idx + distance
        sublayer.qubits_idx = qubits_idx.tolist()

当 $n_{qsc}$ 比较大的时候，我们可以通过对上述的2–局部影子电路进行扩展来搭建 $n_{qsc}$–局部影子电路。我们可以打印一个深度为2的4–局部影子电路来看看这种扩展是怎么进行的：

In [6]:
N = 6
NQSC = 4
D = 2
cir = U_theta(N, n_qsc=NQSC, depth=D)
print(cir)

--Rx(1.162)----Ry(3.081)----Rx(2.008)----*--------------x----Ry(0.401)----*--------------x----Ry(2.556)--
                                         |              |                 |              |               
--Rx(4.303)----Ry(4.531)----Rx(5.933)----x----*---------|----Ry(3.765)----x----*---------|----Ry(4.912)--
                                              |         |                      |         |               
--Rx(5.490)----Ry(5.024)----Rx(0.269)---------x----*----|----Ry(4.286)---------x----*----|----Ry(6.192)--
                                                   |    |                           |    |               
--Rx(6.190)----Ry(3.108)----Rx(6.206)--------------x----*----Ry(1.648)--------------x----*----Ry(2.013)--
                                                                                                         
---------------------------------------------------------------------------------------------------------
                                              

### 影子特征

在前面的教程中，我们多次提到影子特征，那么究竟什么是影子特征呢？我们可以认为它是一个从希尔伯特空间到经典空间的投影。有非常多这样的投影都可以作为影子特征。这里我们将在 $n_{qsc}$ 个量子比特上用泡利 $X\otimes X\otimes \cdots \otimes X$ 算符进行测量得到的期望值作为影子特征。在我们前面举的 $n=4$ 的例子中，在前两个量子比特上测量得到的 $O_1 = \left<X\otimes X\right>$ 就是我们用影子电路获取的第一个影子特征。

In [7]:
# 构造用来获取影子特征的可观测量
def observable(n_start, n_qsc=2):
    pauli_str = ','.join('x' + str(i) for i in range(n_start, n_start + n_qsc))
    hamiltonian = paddle_quantum.Hamiltonian([[1.0, pauli_str]])

    return hamiltonian

### 用 FCNN 进行经典后处理

我们将获得的所有影子特征传入一个经典的 FCNN，并使用归一化指数函数作为激活函数。经过归一化指数函数后得到的输出是一个概率分布，输出中的第 $i$ 个元素代表这个数据点属于第 $i$ 类的概率，我们认为预测的类别就是输出中概率最高的类别。为了提高预测的准确率，我们将预测标签和实际标签之间的累计差距作为损失函数进行优化：

$$
\mathcal{L}(\mathbf{\theta}, \mathbf{W}, \mathbf{b}) = -\frac{1}{N}\sum\limits_{i=1}^{N}\sum\limits_{j=1}^{k}y^i_j\log{\tilde{y}^i_j}.\tag{2}
$$

In [8]:
class Net(paddle.nn.Layer):
    def __init__(self,
                 n, # 全局量子比特的数量: n
                 n_qsc, # 影子电路的宽度
                 depth=1 # 电路深度
                ):
        super(Net, self).__init__()
        self.n = n
        self.n_qsc = n_qsc
        self.depth = depth
        self.cir = U_theta(self.n, n_qsc=self.n_qsc, depth=self.depth)
        # FCNN, 采用高斯分布来初始化权重和偏置
        self.fc = paddle.nn.Linear(n - n_qsc + 1, 2,
                                   weight_attr=paddle.ParamAttr(initializer=paddle.nn.initializer.Normal()),
                                   bias_attr=paddle.ParamAttr(initializer=paddle.nn.initializer.Normal()))

    # 定义前向传播机制、计算损失函数 和交叉验证正确率
    def forward(self, batch_in, label):
        # 量子部分
        dim = len(batch_in)
        features = []
        for state in batch_in:
            _state = paddle_quantum.State(state)
            f_i = []
            for st in range(self.n - self.n_qsc + 1):
                ob = observable(st, n_qsc=self.n_qsc)
                # 滑动电路使其依次作用在各量子比特上
                slide_circuit(self.cir, 1 if st != 0 else 0)
                expecval = paddle_quantum.loss.ExpecVal(ob)
                out_state = self.cir(_state)
                # 计算期望值
                f_ij = expecval(out_state)
                f_i.append(f_ij)
            # 滑动电路至初始位置
            slide_circuit(self.cir, -st)
            f_i = paddle.concat(f_i)
            features.append(f_i)
        features = paddle.stack(features)
        # 经典部分
        outputs = self.fc(features)
        outputs = F.log_softmax(outputs)
        # 计算损失函数和准确率
        loss = -paddle.mean(paddle.sum(outputs * label)) 
        is_correct = 0
        for i in range(dim):
            if paddle.argmax(label[i], axis=-1) == paddle.argmax(outputs[i], axis=-1):
                is_correct = is_correct + 1
        acc = is_correct / dim

        return loss, acc

In [9]:
def ShadowClassifier(N=4, n_qsc=2, D=1, EPOCH=4, LR=0.1, BATCH=1, N_train=1000, N_test=100):
    # 加载数据
    x_train, y_train, x_test, y_test = data_loading(n_train=N_train, n_test=N_test)
    # 初始化神经网络
    net = Net(N, n_qsc, depth=D)
    # 一般来说，我们利用 Adam 优化器来获得相对好的收敛
    # 当然你可以改成 SGD 或者是 RMSprop
    opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

    # 优化循环
    for ep in range(EPOCH):
        for itr in range(N_train // BATCH):
            # 前向传播计算损失函数和准确率
            loss, batch_acc = net(x_train[itr * BATCH:(itr + 1) * BATCH],
                                  y_train[itr * BATCH:(itr + 1) * BATCH])
            # 反向传播极小化损失函数
            loss.backward()
            opt.minimize(loss)
            opt.clear_grad()
            # 在测试集上测试
            if itr % 10 == 0:
                # 计算测试集上的正确率 test_acc
                loss_useless, test_acc = net(x_test[0:N_test],
                                             y_test[0:N_test])
                # 打印测试结果
                print("epoch:%3d" % ep, "  iter:%3d" % itr,
                      "  loss: %.4f" % loss.numpy(),
                      "  batch acc: %.4f" % batch_acc,
                      "  test acc: %.4f" % test_acc)

接下来我们来看看实际的训练效果，整个训练过程大概需要八分钟：

In [11]:
time_st = time.time()
ShadowClassifier(
    N=10,          # 全局量子比特的数量: n 
    n_qsc=2,       # 影子电路的宽度
    D=1,           # 采用的电路深度
    EPOCH=3,       # 训练 epoch 轮数
    LR=0.02,       # 设置学习速率
    BATCH=20,      # 训练时 batch 的大小
    N_train=500,   # 规定训练集大小
    N_test=100     # 规定测试集大小
)
print("time used:", time.time() - time_st)

/var/folders/8_/kmzpk4lx35x7t6zp95sm5j100000gn/T/ipykernel_56443/103109265.py:6: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_dataset = np.array([i for i in train_dataset if i[1][0] == 0 or i[1][0] == 1], dtype=object)
/var/folders/8_/kmzpk4lx35x7t6zp95sm5j100000gn/T/ipykernel_56443/103109265.py:7: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.a

epoch:  0   iter:  0   loss: 20.8973   batch acc: 0.4500   test acc: 0.5700
epoch:  0   iter: 10   loss: 10.2715   batch acc: 0.8500   test acc: 0.7600
epoch:  0   iter: 20   loss: 8.8303   batch acc: 0.9000   test acc: 0.9300
epoch:  1   iter:  0   loss: 7.1990   batch acc: 1.0000   test acc: 0.9300
epoch:  1   iter: 10   loss: 6.4360   batch acc: 1.0000   test acc: 0.9500
epoch:  1   iter: 20   loss: 6.7546   batch acc: 1.0000   test acc: 0.9700
epoch:  2   iter:  0   loss: 5.5871   batch acc: 1.0000   test acc: 0.9700
epoch:  2   iter: 10   loss: 4.2780   batch acc: 1.0000   test acc: 0.9800
epoch:  2   iter: 20   loss: 4.7867   batch acc: 1.0000   test acc: 0.9800
time used: 335.4554250240326


## 总结

VSQL 是一个基于经典影子的量子–经典混合算法，它以卷积的方式获取局部影子特征。VSQL 结合了参数化量子电路 $U(\mathbf{\theta})$ 和经典 FCNN，在二分类任务中表现出色。在[量子分类器](./QClassifier_CN.ipynb)的教程里，我们介绍了一种常用的使用参数化量子电路的分类器。这种分类器在所有的量子比特上作用参数化电路，所以优化器必须搜索整个希尔伯特空间来找到最优的 $\mathbf{\theta}$。和这种方法不同的是，VSQL 每次只在少量选中的量子比特上作用参数化局部量子电路 $U(\mathbf{\theta})$。对于一个 $k$ 分类问题来说，VSQL 需要训练的是参数化局部量子电路中的参数和经典 FCNN 中的参数，加在一起总共是 $n_{qsc}D + [(n-n_{qsc}+1)+1]k$ 个参数。我们可以看到，和常用的变分量子分类器需要训练的参数个数 $nD$ 比起来，VSQL 参数化局部量子电路部分需要训练的参数数量 $n_{qsc}D$ 大大减少，也就是说，VSQL 需要的量子资源大大减少。

---

## 参考文献

[1] Li, Guangxi, Zhixin Song, and Xin Wang. "VSQL: Variational Shadow Quantum Learning for Classification." [Proceedings of the AAAI Conference on Artificial Intelligence. Vol. 35. No. 9. 2021.](https://ojs.aaai.org/index.php/AAAI/article/view/17016)

[2] Goodfellow, Ian, et al. Deep learning. Vol. 1. No. 2. Cambridge: MIT press, 2016.

[3] LeCun, Yann. "The MNIST database of handwritten digits." http://yann.lecun.com/exdb/mnist/ (1998).